In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

#import yfinance as yf

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## Set start date variable - dataframes will be created starting from this date
start_date = '2021-01-01'
end_date = '2021-09-30'
default_range = '10m' ## Default Range for IEX functions - don't need more at the moment

In [3]:
ftd_df = pd.read_csv(
    Path('../Resources/ftd_all_data.csv'),
    index_col=0, parse_dates=True
)
#ftd_df

In [4]:
symbol_df = pd.read_csv(
    Path('../Resources/symbol_all_list.csv'),
    index_col=0
)
#symbol_df

In [5]:
# new_data = ftd_df
# new_data = new_data.reset_index()
# new_data = new_data.set_index("CUSIP")
# test_csn = "36467W109"
# new_data = new_data.loc[test_csn]
# new_data

In [6]:
def return_ftd_data_cusip(cusip_number):
    df = ftd_df
    df = df.reset_index()
    df = df.set_index("CUSIP")
    df = df.loc[cusip_number]
    df = df.set_index('Date')
    return df

In [7]:
# ## Testing 
# tsla_cusip = "88160R101" 
# aapl_cusip = '037833100'
# gme_cusip = "36467W109"
# test_ftd_df = return_ftd_data_cusip(aapl_cusip)
# test_ftd_df

In [8]:
## IEX Functions and Setup

In [9]:
## IEX Setup and Test 
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

real_token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [10]:
def get_chart(stock_ticker, chart_range=default_range,token=False):
    
    # Token = False means using Test Token 
    if token == False:        
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    # Token = True means using Real Token and Real API calls 
    elif token == True:
        resp_data = requests.get(base_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+real_token)
        
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df 

def get_concise_data(stock_ticker, chart_range=default_range,token=False):
    #date_range = chart_range
    df = get_chart(stock_ticker,chart_range,token=token)
    df = df[['close','high','low','open','symbol','volume','change','changePercent']]
    df = df[start_date:end_date]  ## Dates declared at start of file 
    return df

In [11]:
# gme_ftd = return_ftd_data_cusip(gme_cusip)
# gme_data = get_concise_data('GME')
# ftd_data = pd.DataFrame(gme_ftd)
# year_data = gme_data['2021-01':'2021-09']
# gme_data

In [12]:
# gme_ftd

In [13]:
# frames = [df1,df2]
# df_keys = pd.concat(frames, keys=['x', 'y'])

# df_keys

In [14]:
# iex_data = pd.DataFrame(year_data)
# ftd_data = pd.DataFrame(gme_ftd)

# iex_data.drop(columns={'symbol'},inplace=True)
# ftd_data.drop(columns={'SYMBOL'},inplace=True)
# iex_data

In [15]:
# ftd_data

In [16]:
# df_merge_col = pd.merge(iex_data, ftd_data, on='Date',how='outer')
# df_merge_col['QUANTITY_FAILS'] = df_merge_col['QUANTITY_FAILS'].fillna(0)
# df_merge_col

In [17]:
# df_merge_col.info()

In [18]:
# test_err = pd.DataFrame(df_merge_col)
# nullcolumn = test_err['QUANTITY_FAILS']
# nullcolumn = nullcolumn.reset_index(drop=True)
# nullcolumn

In [19]:
#test_err.reset_index(inplace=True)

# length_ = len(nullcolumn)
## This never worked 
# for i in range(length_):
#     var = nullcolumn[i]
#     if var == NaN:
#         print(i)
#     else:
#         continue

In [20]:
# # df['DataFrame Column'] = df['DataFrame Column'].fillna(0)
# test_err['QUANTITY_FAILS'] = test_err['QUANTITY_FAILS'].fillna(0)
# test_err.tail(50)

# Found NaN - now 0.0 as it should be 

In [21]:
## Make a Function that does all 

def get_ftd_and_iex_for_2021(cusip,symbol,token1=False):
    ftd_data = return_ftd_data_cusip(cusip)
    iex_data = get_concise_data(symbol,token=token1)
    #year_data = iex_data[start_date:end_date]  ## Dates declared at start of file 
    
#     iex_data = pd.DataFrame(year_data)
#     ftd_data = pd.DataFrame(gme_ftd)

    #iex_data.drop(columns={'symbol'},inplace=True)
    ftd_data.drop(columns={'SYMBOL'},inplace=True)
    
    df = pd.merge(iex_data, ftd_data, on='Date',how='outer')
    df['QUANTITY_FAILS'] = df['QUANTITY_FAILS'].fillna(0)
    return df

In [22]:
index1 = 250
index2 = 254
test_symbol1 = symbol_df.iloc[index1]['SYMBOL']
test_cusip1 = symbol_df.iloc[index1]['CUSIP']

print(test_symbol1,test_cusip1)

ACVF 26923N108


In [23]:
test_symbol2 = symbol_df.iloc[index2]['SYMBOL']
test_cusip2 = symbol_df.iloc[index2]['CUSIP']
print(test_symbol2,test_cusip2)

ACWV 464286525


In [24]:
# test_data = get_concise_data(test_symbol)
# test_data

In [25]:
test_return_df1 = get_ftd_and_iex_for_2021(test_cusip1,test_symbol1)
test_return_df1

,close,high,low,open,symbol,volume,change,changePercent,QUANTITY_FAILS
Date,,,,,,,,,
2021-01-04,27.9395,29.0300,27.9395,28.690,ACVF,3813,-0.494925,-0.0175,541.0
2021-01-05,27.9400,28.7500,27.9400,28.370,ACVF,3795,0.218318,0.0078,229.0
2021-01-06,28.0565,28.6800,27.7720,28.391,ACVF,3039,0.151074,0.0054,8.0
2021-01-07,28.9794,28.9794,27.9000,27.900,ACVF,1050,0.454893,0.0162,0.0
2021-01-08,28.5658,29.1400,28.1030,28.870,ACVF,4694,0.073442,0.0027,9.0
...,...,...,...,...,...,...,...,...,...
2021-09-24,33.0540,34.1500,33.0540,34.150,ACVF,3080,-0.037214,-0.0011,4564.0
2021-09-27,34.3400,34.3400,33.6900,33.740,ACVF,19209,-0.129708,-0.0039,5913.0
2021-09-28,33.3300,33.8200,33.3300,33.730,ACVF,11050,-0.665020,-0.0203,415.0


In [26]:
test_return_df2 = get_ftd_and_iex_for_2021(test_cusip2,test_symbol2)
test_return_df2

# test_df = get_concise_data(test_symbol2)
# test_df

,close,high,low,open,symbol,volume,change,changePercent,QUANTITY_FAILS
Date,,,,,,,,,
2021-01-04,97.50,101.0500,97.5000,101.05,ACWV,287126,-0.522784,-0.0053,0.0
2021-01-05,100.40,101.1365,96.7600,96.97,ACWV,343731,0.500000,0.0053,0.0
2021-01-06,100.84,101.1000,96.8100,96.81,ACWV,405707,0.500786,0.0050,0.0
2021-01-07,99.78,100.5800,98.9351,100.58,ACWV,413880,-0.020983,-0.0002,0.0
2021-01-08,102.66,102.6600,98.8200,98.82,ACWV,489913,0.666648,0.0069,9165.0
...,...,...,...,...,...,...,...,...,...
2021-09-24,106.50,108.8600,105.6200,108.86,ACWV,82056,-0.233057,-0.0023,1163.0
2021-09-27,109.03,109.0300,108.5400,108.93,ACWV,112400,-0.796442,-0.0078,0.0
2021-09-28,107.75,107.7500,103.9200,103.92,ACWV,156702,-1.602796,-0.0160,1245.0


In [27]:
frames = [test_return_df1,test_return_df2]
df_keys = pd.concat(frames, keys=[test_symbol1, test_symbol2])

df_keys

close      high       low     open symbol  volume  \
     Date                                                               
ACVF 2021-01-04   27.9395   29.0300   27.9395   28.690   ACVF    3813   
     2021-01-05   27.9400   28.7500   27.9400   28.370   ACVF    3795   
     2021-01-06   28.0565   28.6800   27.7720   28.391   ACVF    3039   
     2021-01-07   28.9794   28.9794   27.9000   27.900   ACVF    1050   
     2021-01-08   28.5658   29.1400   28.1030   28.870   ACVF    4694   
...                   ...       ...       ...      ...    ...     ...   
ACWV 2021-09-24  106.5000  108.8600  105.6200  108.860   ACWV   82056   
     2021-09-27  109.0300  109.0300  108.5400  108.930   ACWV  112400   
     2021-09-28  107.7500  107.7500  103.9200  103.920   ACWV  156702   
     2021-09-29  106.1300  106.9000  103.9000  106.800   ACWV   64896   
     2021-09-30  104.9800  107.9000  103.2300  105.720   ACWV   94468   

                   change  changePercent  QUANTITY_FAILS  
     Date                                                 
ACVF 2021-01-04 -0.494925        -0.0175           541.0  
     2021-01-05  0.218318         0.0078           229.0  
     2021-01-06  0.151074         0.0054             8.0  
     2021-01-07  0.454893         0.0162             0.0  
     2021-01-08  0.073442         0.0027             9.0  
...                   ...            ...             ...  
ACWV 2021-09-24 -0.233057        -0.0023          1163.0  
     2021-09-27 -0.796442        -0.0078             0.0  
     2021-09-28 -1.602796        -0.0160          1245.0  
     2021-09-29  0.470223         0.0045           101.0  
     2021-09-30 -0.607033        -0.0057             0.0  

[376 rows x 9 columns]

In [28]:
df_keys.loc['ACWV']

,close,high,low,open,symbol,volume,change,changePercent,QUANTITY_FAILS
Date,,,,,,,,,
2021-01-04,97.50,101.0500,97.5000,101.05,ACWV,287126,-0.522784,-0.0053,0.0
2021-01-05,100.40,101.1365,96.7600,96.97,ACWV,343731,0.500000,0.0053,0.0
2021-01-06,100.84,101.1000,96.8100,96.81,ACWV,405707,0.500786,0.0050,0.0
2021-01-07,99.78,100.5800,98.9351,100.58,ACWV,413880,-0.020983,-0.0002,0.0
2021-01-08,102.66,102.6600,98.8200,98.82,ACWV,489913,0.666648,0.0069,9165.0
...,...,...,...,...,...,...,...,...,...
2021-09-24,106.50,108.8600,105.6200,108.86,ACWV,82056,-0.233057,-0.0023,1163.0
2021-09-27,109.03,109.0300,108.5400,108.93,ACWV,112400,-0.796442,-0.0078,0.0
2021-09-28,107.75,107.7500,103.9200,103.92,ACWV,156702,-1.602796,-0.0160,1245.0


In [29]:
test_key_df = get_ftd_and_iex_for_2021(test_cusip1,test_symbol1)
test_key_df

my_dict = {}
my_dict[test_symbol1] = test_key_df
my_dict

{'ACVF':               close     high      low    open symbol  volume    change  \
 Date                                                                     
 2021-01-04  28.0132  29.4200  27.6000  29.420   ACVF    3887 -0.481690   
 2021-01-05  28.4600  28.4600  27.6488  28.370   ACVF    3837  0.216510   
 2021-01-06  29.0766  29.0766  28.1940  28.194   ACVF    3048  0.152315   
 2021-01-07  28.9008  29.4490  27.9000  28.000   ACVF    1008  0.456882   
 2021-01-08  29.3562  29.5700  29.0520  29.570   ACVF    4747  0.073921   
 ...             ...      ...      ...     ...    ...     ...       ...   
 2021-09-24  33.1972  34.2050  33.1972  33.500   ACVF    3110 -0.037146   
 2021-09-27  33.7500  34.2800  33.6400  34.280   ACVF   18890 -0.125272   
 2021-09-28  33.3700  34.3000  32.9600  33.750   ACVF   10968 -0.648857   
 2021-09-29  32.7000  33.6100  32.7000  33.610   ACVF    9527  0.020981   
 2021-09-30  34.0000  34.0000  32.9700  32.970   ACVF    7655 -0.311851   
 
             ch

In [30]:
test_key_df2 = get_ftd_and_iex_for_2021(test_cusip2,test_symbol2)
my_dict[test_symbol2] = test_key_df2
my_dict

{'ACVF':               close     high      low    open symbol  volume    change  \
 Date                                                                     
 2021-01-04  28.0132  29.4200  27.6000  29.420   ACVF    3887 -0.481690   
 2021-01-05  28.4600  28.4600  27.6488  28.370   ACVF    3837  0.216510   
 2021-01-06  29.0766  29.0766  28.1940  28.194   ACVF    3048  0.152315   
 2021-01-07  28.9008  29.4490  27.9000  28.000   ACVF    1008  0.456882   
 2021-01-08  29.3562  29.5700  29.0520  29.570   ACVF    4747  0.073921   
 ...             ...      ...      ...     ...    ...     ...       ...   
 2021-09-24  33.1972  34.2050  33.1972  33.500   ACVF    3110 -0.037146   
 2021-09-27  33.7500  34.2800  33.6400  34.280   ACVF   18890 -0.125272   
 2021-09-28  33.3700  34.3000  32.9600  33.750   ACVF   10968 -0.648857   
 2021-09-29  32.7000  33.6100  32.7000  33.610   ACVF    9527  0.020981   
 2021-09-30  34.0000  34.0000  32.9700  32.970   ACVF    7655 -0.311851   
 
             ch

In [31]:
my_dict['ACWV']

,close,high,low,open,symbol,volume,change,changePercent,QUANTITY_FAILS
Date,,,,,,,,,
2021-01-04,99.90,100.9000,99.680,100.90,ACWV,287884,-0.510703,-0.0054,0.0
2021-01-05,99.20,99.2000,97.350,98.76,ACWV,353945,0.500000,0.0054,0.0
2021-01-06,99.77,102.2000,97.390,97.39,ACWV,416534,0.497278,0.0050,0.0
2021-01-07,98.78,100.3843,98.240,98.24,ACWV,407842,-0.020386,-0.0002,0.0
2021-01-08,99.90,101.6900,98.576,101.69,ACWV,473526,0.664262,0.0068,9165.0
...,...,...,...,...,...,...,...,...,...
2021-09-24,107.39,110.3100,107.040,107.13,ACWV,82218,-0.237163,-0.0023,1163.0
2021-09-27,109.04,109.0400,107.080,107.08,ACWV,111297,-0.802772,-0.0076,0.0
2021-09-28,106.24,107.6100,104.930,106.59,ACWV,158653,-1.582813,-0.0150,1245.0
